In [17]:
from pymongo import MongoClient
from pprint import pprint
from bson.objectid import ObjectId
from collections import defaultdict
import dateutil
import json
import requests

In [2]:
with open("secrets.json", "r") as secrets:
    mongo_uri = json.load(secrets)["mongo_uri"]

In [70]:
client = MongoClient(mongo_uri)
db=client.aggie

In [4]:
# testing connection
facebook = db.reports.count_documents({'$and': [{'_sourceNicknames': ["ct"]}]})
print(facebook)

225289


In [10]:
image_text_reports_count = db.reports.count_documents({'$and':[
    {"metadata.imageText": {'$ne': None}}, 
    {'fetchedAt':{'$lte': dateutil.parser.parse('2020-12-07 16:24:28.0000')}}, 
    {'$or' : [{"_sourceNicknames": ["ct"]}, {"_sourceNicknames": ["fb"]}]}]})
print("Image Text Reports : {}".format(image_text_reports_count))

Image Text Reports : 7857


In [13]:
image_text_reports = db.reports.find({'$and':[
    {"metadata.imageText": {'$ne': None}}, 
    {'fetchedAt':{'$lte': dateutil.parser.parse('2020-12-07 16:24:28.0000')}}, 
    {'$or' : [{"_sourceNicknames": ["ct"]}, {"_sourceNicknames": ["fb"]}]}]}, {"metadata.imageText": 1, "_id": 1, "metadata.externalUrl": 1}).sort([("fetchedAt", 1)])
image_text_reports = list(image_text_reports)
print("Image text reports : {}".format(len(image_text_reports)))

Image text reports : 7857


In [66]:
# with open("image_text_reports.json", "w") as out:
#     json.dump(image_text_reports, out, indent = 4,default=str, ensure_ascii=False)

In [65]:
# url = "http://localhost:5000/predict"

# for index in range(len(image_text_reports)):
#     report = image_text_reports[index]
#     payload = {"inputData" : report["metadata"]["imageText"]}
#     headers = {
#         'Content-Type': "application/json",
#         'Cache-Control': "no-cache",
#         'Postman-Token': "3d38908c-9e04-4dab-9a2d-eddadf26ca44"
#         }
#     response = requests.request("POST", url, data=json.dumps(payload, ensure_ascii=False).encode('utf-8'), headers=headers)
#     response = json.loads(response.text,  encoding='utf-8')
#     image_text_reports[index]["metadata"]["hateSpeechScore"] = response["result"]["hateSpeechScore"] 
#     if index % 500 == 0:
#         with open("processed_image_text_reports.json", "w") as out:
#             json.dump(image_text_reports, out, indent = 4,default=str, ensure_ascii=False)
    

In [67]:
# with open("processed_image_text_reports.json", "w") as out:
#     json.dump(image_text_reports, out, indent = 4,default=str, ensure_ascii=False)

In [68]:
# for index in range(len(image_text_reports)):
#     if image_text_reports[index]["metadata"]["hateSpeechScore"] == 0:
#         image_text_reports[index]["metadata"]["hateSpeechScore"] = None
# with open("processed_image_text_reports.json", "w") as out:
#     json.dump(image_text_reports, out, indent = 4,default=str, ensure_ascii=False)

In [76]:
# count = 0
# for report in image_text_reports:
    # db.reports.update_one({"_id": ObjectId(report["_id"])}, {'$set' : {"metadata.hateSpeechScore": report["metadata"]["hateSpeechScore"]}})
    # if count % 500 == 0:
        # print(count)

In [89]:
image_text_reports_len = db.reports.count_documents({'$and':[
    {"metadata.imageText": {'$ne': None}}, 
    {'fetchedAt':{'$lte': dateutil.parser.parse('2020-12-07 16:24:28.0000')}}, 
    {'$or' : [{"_sourceNicknames": ["ct"]}, {"_sourceNicknames": ["fb"]}]},
    {'metadata.hateSpeechScore': {'$ne': None}}]})
print("Image text reports : {}".format(image_text_reports_len))

Image text reports : 4687


In [90]:
image_text_reports_hs_45 = db.reports.count_documents({'$and':[
    {"metadata.imageText": {'$ne': None}}, 
    {'fetchedAt':{'$lte': dateutil.parser.parse('2020-12-07 16:24:28.0000')}}, 
    {'$or' : [{"_sourceNicknames": ["ct"]}, {"_sourceNicknames": ["fb"]}]},
    {'metadata.hateSpeechScore': {'$gte': 0.45}}]})
image_text_reports_hs_55 = db.reports.count_documents({'$and':[
    {"metadata.imageText": {'$ne': None}}, 
    {'fetchedAt':{'$lte': dateutil.parser.parse('2020-12-07 16:24:28.0000')}}, 
    {'$or' : [{"_sourceNicknames": ["ct"]}, {"_sourceNicknames": ["fb"]}]},
    {'metadata.hateSpeechScore': {'$gte': 0.55}}]})
image_text_reports_hs_65 = db.reports.count_documents({'$and':[
    {"metadata.imageText": {'$ne': None}}, 
    {'fetchedAt':{'$lte': dateutil.parser.parse('2020-12-07 16:24:28.0000')}}, 
    {'$or' : [{"_sourceNicknames": ["ct"]}, {"_sourceNicknames": ["fb"]}]},
    {'metadata.hateSpeechScore': {'$gte': 0.65}}]})
print("Image text reports HS 0.45: {} ({})".format(image_text_reports_hs_45, image_text_reports_hs_45/image_text_reports_len))
print("Image text reports HS 0.55: {} ({})".format(image_text_reports_hs_55, image_text_reports_hs_55/image_text_reports_len))
print("Image text reports HS 0.65: {} ({})".format(image_text_reports_hs_65, image_text_reports_hs_65/image_text_reports_len))

Image text reports HS 0.45: 3618 (0.7719223383827608)
Image text reports HS 0.55: 2562 (0.5466183059526349)
Image text reports HS 0.65: 86 (0.01834862385321101)


In [91]:
#creating a list of HS incidents
hs_incidents_1_report_list = list(db.incidents.find({'$and': [{'totalReports': 1}, {"title" : {'$regex' : ".*[hH]ate.*"}}, {"storedAt": {'$lte': dateutil.parser.parse('2020-12-07 16:24:28.0000')}}]}))
hs_incidents_gt_1_reports_list = list(db.incidents.find({'$and': [{'totalReports':{'$gt': 1}}, {"title" : {'$regex' : ".*[hH]ate.*"}}, {"storedAt": {'$lte': dateutil.parser.parse('2020-12-07 16:24:28.0000')}}]}))
hs_incidents_list = hs_incidents_1_report_list + hs_incidents_gt_1_reports_list
print(len(hs_incidents_list))

579


In [92]:
#creating a list of hs incident ids for cross referencing
hs_incidents_ids = [str(incident["_id"]) for incident in hs_incidents_list]

In [105]:
non_null_incidents_reports_img_text = db.reports.count_documents({'$and': [{"metadata.imageText": {'$ne': None}},{"metadata.hateSpeechScore":{'$ne' : None}}, {"_incident":{'$exists':True}}, {"_incident": {'$nin': [None, ""]}}, {"fetchedAt": {'$lte': dateutil.parser.parse('2020-12-07 16:24:28.0000')}}]})

non_null_incidents_reports_img_text_list = list(db.reports.find({'$and': [{"metadata.imageText": {'$ne': None}}, {"metadata.hateSpeechScore":{'$ne' : None}},{"_incident":{'$exists':True}}, {"_incident": {'$nin': [None, ""]}}, {"fetchedAt": {'$lte': dateutil.parser.parse('2020-12-07 16:24:28.0000')}}]}))


print("Non null incident field in reports with image text : {}".format(non_null_incidents_reports_img_text))

Non null incident field in reports with image text : 71


In [106]:
#creating a incident ->reports dict
incident_report_dict = defaultdict(list)
for report in non_null_incidents_reports_img_text_list:
    incident_report_dict[report["_incident"]].append(report)
print("Unique incidents created between the timeframe : {}".format(len(incident_report_dict.keys())))

Unique incidents created between the timeframe : 49


In [107]:
# finding intersection between HS incidents and reports that have incidents after HS activation
report_incidents_ids = list(incident_report_dict.keys())
hs_reports_incidents_ids =  list((set(report_incidents_ids) & set(hs_incidents_ids)))
print("# of HS incidents created in the timeframe that are referenced in the reports collection : {}".format(len(hs_reports_incidents_ids)))

# of HS incidents created in the timeframe that are referenced in the reports collection : 7


In [108]:
reports_in_hs_incidents = [v for k, v in incident_report_dict.items() if k in hs_reports_incidents_ids]
print(len(reports_in_hs_incidents))

7


In [109]:
reports_in_hs_incidents_dcntr = []  #flattening
for reports in reports_in_hs_incidents:
    reports_in_hs_incidents_dcntr.extend(reports)

print("Reports added to HS incidents after HS activation : {}".format(len(reports_in_hs_incidents_dcntr)))

Reports added to HS incidents after HS activation : 8


In [110]:
reports_in_hs_incidents_with_burmese = [r for r in reports_in_hs_incidents_dcntr if "hateSpeechScore" in r["metadata"] and r["metadata"]["hateSpeechScore"] is not None]
print("Classifiable reports with Burmese content : {}".format(len(reports_in_hs_incidents_with_burmese)))

Classifiable reports with Burmese content : 8


In [111]:
classifiable_reports_in_hs_incidents_45 = [r for r in reports_in_hs_incidents_with_burmese if r["metadata"]["hateSpeechScore"] >= 0.45]
print("Classifiable reports with Burmese content in HS incidents with HS score > 0.45: {} ({})".format(len(classifiable_reports_in_hs_incidents_45), len(classifiable_reports_in_hs_incidents_45)/len(reports_in_hs_incidents_with_burmese)))

Classifiable reports with Burmese content in HS incidents with HS score > 0.45: 7 (0.875)
